<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/generation/generating_fic_2epoch_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [ ]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [ ]:
!huggingface-cli login

# Loading the model

Redownloading the model required some tweaks, which are described in:
https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-bnc-baby-fictional-2-epochs'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

# Extracting Word Counts and Prompts

In [ ]:
import pandas as pd
import numpy as np
import random
import nltk
from iteround import saferound
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')

In [ ]:
df.head()

In [ ]:
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

In [ ]:
df['sentences'] = df['text'].apply(split_into_sentences)

In [ ]:
df

In [ ]:
random_state = 12
random_sentence_state = 21
sample_count = 20

In [ ]:
def get_random_sentence(sentences_list, random_state=random_sentence_state, min_word_count=5):
    random.seed(random_state)
    filtered_sentences = [sentence for sentence in sentences_list if len(word_tokenize(sentence)) > min_word_count]
    return random.choice(filtered_sentences) if filtered_sentences else None

In [ ]:
df['prompt'] = df['sentences'].apply(get_random_sentence)

In [ ]:
def get_samples(df, n=sample_count, state=random_state):
  return df.sample(n=n, random_state=state)

In [ ]:
df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]

In [ ]:
df_news

In [ ]:
aca_prompts = df_aca['prompt'].to_list()
fic_prompts = df_fic['prompt'].to_list()
news_prompts = df_news['prompt'].to_list()

# Generating output

In [ ]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
#the only difference is the implementation of a deviation value
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
max_length = 1000

In [ ]:
#from Salminen et al.; a seed value was set to allow the generation to be re-produced and continued over multiple sessions
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

The generating loop

In [ ]:
def generate(prompts):
  outputs = []
  for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    random_seed(seed_value=random_state, use_cuda=True)
    o = model.generate(**inputs, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token, max_length=max_length)
    outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))
  return outputs

In [ ]:
aca_outputs = generate(aca_prompts)

In [ ]:
fic_outputs = generate(fic_prompts)

In [ ]:
news_outputs = generate(news_prompts)

In [ ]:
aca_results = pd.DataFrame(aca_outputs, columns=['output'])
aca_results['prompt_category'] = 'ACA'
aca_results['prompt'] = aca_prompts

fic_results = pd.DataFrame(fic_outputs, columns=['output'])
fic_results['prompt_category'] = 'FIC'
fic_results['prompt'] = fic_prompts

news_results = pd.DataFrame(news_outputs, columns=['output'])
news_results['prompt_category'] = 'NEWS'
news_results['prompt'] = news_prompts

result_frames = [aca_results, fic_results, news_results]

result_df = pd.concat(result_frames, ignore_index=True)
result_df

In [ ]:
result_df.to_csv('/content/drive/MyDrive/Linguistic Data Science/data/2epoch-fic-output.csv', index=False)